In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker



In [2]:
#Paimam years į konstanta , ir paimam mums reikiama url
years =list(range(2001 ,2022))
url_data ="https://admin.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E{}&cat=Score&agg=Accumulated"


In [3]:
#Pasiimam gautus html failus ir susidedame juos į mums reikiamą folderį

def receive_htmls_data_from_web():
  for year in years :
   url = url_data.format(year)
   data = requests.get(url)
  with open("admin_euroleague/{}.html".format(year), "w+" , encoding="utf-8") as file :
    file.write(data.text)
    return file 
    

In [4]:
#Pasiimam mus dominantį html pagal metus , ir pagal lentelės ID ją išsitraukiam priskirdami kintamajam
def get_wanted_table(file):
 with open ("admin_euroleague/2001.html",encoding="utf-8") as f :
    page =f.read()
    soup = BeautifulSoup(page, "html.parser")
    soup.find('tr',class_="stats-results")
    statistics_table=soup.find(id="ctl00_ctl00_ctl00_ctl00_maincontainer_maincontent_contentpane_ctl01_stats_results")
    leaders_2001= pd.read_html(str(statistics_table))[0]
    return leaders_2001
 

In [8]:
# Susikuriame ryšį su duomenų baze kitame faile 
engine = create_engine('sqlite:///scorers_01.db')
Session = sessionmaker(bind=engine)
session = Session()
      
    

In [9]:
def put_data_into_table(leaders_2001):
 leaders_2001.to_sql(
    name = "scoring_leaders_01" ,
    con = engine , 
    index = False ,
    if_exists = "append"
)

  

In [12]:
leaders_2001_select = session.execute('select * from "scoring_leaders_01"')
leaders_2001_select.all()
print(leaders_2001_select)


In [16]:
def get_scorers_from_all_tables():
    dfs = []
    for year in years :
        with open ("admin_euroleague/{}.html".format(year),encoding="utf-8") as f :
            page =f.read()
            soup = BeautifulSoup(page, "html.parser")
            soup.find('tr',class_="StatsHeaderGridResults")
            statistics_table=soup.find(id="ctl00_ctl00_ctl00_ctl00_maincontainer_maincontent_contentpane_ctl01_ctl01_gvResults")
            leaders_2001= pd.read_html(str(statistics_table))[0]
            leaders_2001["Year"] = year
            dfs.append(leaders_2001)
            return dfs

In [18]:
def make_data_visible(dfs):
    leaders = pd.concat(dfs)
   
        
    